In [3]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt

# Paths
data_train_path = r"C:\Users\acer\OneDrive\Desktop\model training\train"
data_val_path = r"C:\Users\mohan\Downloads\model training SAMARTH\model training\valid"
data_test_path = r"C:\Users\acer\OneDrive\Desktop\model training\valid"



# Params
img_width, img_height = 180, 180
img_size = (img_width, img_height)
batch_size = 32  # increased batch size for better batch statistics
num_classes = 1  # binary classification (wet/dry)

# Data augmentation - increased a bit for better generalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.3,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    vertical_flip=True  # added vertical flip since waste images may vary
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# Data loaders
train_data = train_datagen.flow_from_directory(
    data_train_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

val_data = val_test_datagen.flow_from_directory(
    data_val_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

test_data = val_test_datagen.flow_from_directory(
    data_test_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Load base MobileNetV2 model pretrained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Unfreeze last few layers of the base model for fine-tuning (say last 20 layers)
for layer in base_model.layers[:-20]:
    layer.trainable = False
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Build your model with L2 regularization
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),  # increased dropout
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

# Compile with a lower learning rate to avoid large jumps in fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
checkpoint = ModelCheckpoint("best_model.keras", monitor='val_loss', save_best_only=True, verbose=1)

# Train
epochs = 50  # more epochs with early stopping

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=2
)

# Plot Loss & Accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss', color='blue')
plt.plot(history.history['val_loss'], label='Val Loss', color='orange')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy', color='green')
plt.plot(history.history['val_accuracy'], label='Val Accuracy', color='red')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Evaluate on test data
test_loss, test_acc = model.evaluate(test_data)
print(f"\n✅ Test Accuracy: {test_acc*100:.2f}%")

import tensorflow as tf

# Use the same image size as your model input
img_height = 180
img_width = 180

# Step 1: Load image and resize
image_path =r"C:\Users\mohan\Downloads\dry waste.jpeg"
img = tf.keras.utils.load_img(image_path, target_size=(img_height, img_width))

# Step 2: Convert image to array
img_array = tf.keras.utils.img_to_array(img)

# Step 3: Rescale (same as training)
img_array = img_array / 255.0

# Step 4: Add batch dimension
img_batch = tf.expand_dims(img_array, axis=0)

# Step 5: Predict (outputs probability for class 1)
pred = model.predict(img_batch)[0][0]

# Step 6: Decide class based on threshold 0.5
predicted_label = 'wet' if pred >= 0.5 else 'dry'



Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Step 7: Print the result and confidence
print(f"🔍 Predicted Class: {predicted_label} (Confidence: {pred:.2f})")

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array

# === Step 1: Load the saved model ===
model_path = r"C:\Users\acer\Downloads\waste_classifier.h5"
model = load_model(model_path)

# === Step 2: Load and preprocess the image ===
img_path = r"C:\Users\acer\Downloads\dry2.jpg" # Change to test another image
img_height, img_width = 180, 180

# Load and resize image
img = load_img(img_path, target_size=(img_height, img_width))
img_array = img_to_array(img) / 255.0  # Normalize
img_batch = tf.expand_dims(img_array, axis=0)  # Add batch dimension

# === Step 3: Predict ===
pred = model.predict(img_batch)[0][0]  # Get the probability
label = "wet" if pred >= 0.5 else "dry"
confidence = pred if pred >= 0.5 else 1 - pred

# === Step 4: Show result ===
print(f"🔍 Predicted Class: {label.upper()} (Confidence: {confidence:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
🔍 Predicted Class: DRY (Confidence: 1.00)
